<a href="https://colab.research.google.com/github/Yiecheverria/Modelo-ScrapyData/blob/main/Analisis_violaciones_global_ModeloData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ***ModelData***

In [ ]:
# Instalación de las librerias o dependencias necesarias
!pip install beautifulsoup4 requests pandas
# Load dependencies
import json # modulo para trabajar con archivos JSON
import requests # libreria para hacer peticiones HTTP
import pandas as pd # libreria para procesar datos, proporciona funciones para analíticas
from bs4 import BeautifulSoup as bs # Extraer información de paginas web

In [ ]:
# Función que convierte una lista de elementos en un dataframe de pandas
def rowsToDataFrame(rows):
    df = pd.DataFrame(rows[1:], columns=rows[0])
    return df

In [ ]:
import re

# Función para extraer los datos de una tabla HTML
def processTableData(tbl):
    rows = []
    first_row_skipped = False
    flag = False
    skip = 0

    for child in tbl.find('tbody').children: # itera los items internos de la tabla [<tr>]
        row = []

        # Salta la primera fila
        if not first_row_skipped:
            first_row_skipped = True
            continue

        for td in child: # itera los items internos de la tabla [<td>]
            try:
                item = td.text.replace('\n', '') # recupera el texto del elemento [<td>]

                # Utiliza expresiones regulares para quitar texto entre corchetes
                item = re.sub(r'\[.*\]', '', item)

                if skip == 10:
                  continue

                if item == "País/territorio" or item == "Noruega":
                    flag = True

                if flag:
                    if item:
                        row.append(item) # Agrega cada elemento de la fila
                        skip += 1
                else:
                    item = item if item is not None and item.strip() != '' else '0'
                    row.append(item)
                    skip += 1
            except:
                continue

        flag = False
        skip = 0

        if len(row) > 0:
            rows.append(row) # agrega todos los campos de una fila de la tabla

    return rows

In [ ]:
# Función para exxtrar la data de la URL y procesar el contenido HTML
def processDataHTML(data):
    soup = bs(data['sections'][1]['text'], 'html.parser') # Recupera el item del JSON y lo convierte en contenido HTML
    tbl = soup.find_all('table')[0] # Recupera la tabla del contenido HTML
    # print(tbl.prettify())
    tblRows = processTableData(tbl) # Ejecuta la función apra porcesar el contenido de la Tabla
    return tblRows

In [ ]:
# Read website
# r = requests.get('https://www.wikiwand.com/es/Anexo:Tabla_estadística_de_la_Copa_Mundial_de_Fútbol')
r = requests.get('https://www.wikiwand.com/mcs-api/es.wikipedia.org/v1/page/mobile-sections-remaining/Estad%C3%ADstica_de_violaciones')  # Recuperamos el contenido de la web
# print(r)
table = processDataHTML(json.loads(r.content)) # Formate el contenido en formato JSON
df = rowsToDataFrame(table) # ejecuta la funcion para recupera los datos de la tabla como dataframe de Pandas
df

,0,País/territorio,2003,2004,2005,2006,2007,2008,2009,2010
0,0,Kenia,0,1440,1365,1291,876,735,847,0
1,0,Mauricio,53,40,38,64,77,69,57,51
2,0,Mozambique,0,102,64,46,41,47,44,0
3,0,Uganda,0,549,732,750,599,1536,619,709
4,0,Zimbabue,3858,4997,4790,4967,4762,3186,0,0
...,...,...,...,...,...,...,...,...,...,...
117,0,Suiza,547,573,646,639,648,612,666,543
118,0,Australia,0,0,0,0,0,6382,6343,6378
119,0,Nueva Zelanda,0,0,994,1183,1143,1130,1128,1129
120,0,Sudáfrica,0,69117,68076,65201,63818,70514,68332,66196


In [ ]:
# Inspeccionamos el tipo de dato de cada variable
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 122 entries, 0 to 121
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   0                122 non-null    object
 1   País/territorio  122 non-null    object
 2   2003             122 non-null    object
 3   2004             122 non-null    object
 4   2005             122 non-null    object
 5   2006             122 non-null    object
 6   2007             122 non-null    object
 7   2008             122 non-null    object
 8   2009             122 non-null    object
 9   2010             122 non-null    object
dtypes: object(10)
memory usage: 9.7+ KB


In [ ]:
# Guardamos las datos extraidos como archivo CSV
# se puede aplcair otras opciones para guardar (Base de datos)
df.to_csv('Datos_violacion.csv')


In [ ]:
# Especifica la ruta y el nombre de archivo para guardar el archivo Excel
excel_file_path = 'Datos_violacion.xlsx'

# Exporta el DataFrame a un archivo Excel
df.to_excel(excel_file_path, index=False)
